In [68]:
from cleansing import get_integrated_db, read_table_from_db

def all_data_frames(fancy_db):
    days = read_table_from_db(fancy_db, "days")
    delays = read_table_from_db(fancy_db, "delays")
    snapshots = read_table_from_db(fancy_db, "snapshot")
    candidate_trains = read_table_from_db(fancy_db, "candidate_trains")
    weather = read_table_from_db(fancy_db, "weather")
    return days, delays, snapshots, candidate_trains, weather

days, delays, snapshots, candidate_trains, weather = get_integrated_db(all_data_frames)

In [69]:
weather["date"] = pd.to_datetime(weather["at_time"]).apply(lambda row: row.date())
weather["time"] = pd.to_datetime(weather["at_time"]).apply(lambda row: row.time())
days.day = pd.to_datetime(days.day)

import datetime

def on_point(hour, df):
    weather_on_point = weather[weather["time"] == datetime.time(hour, 0)].drop(columns=["at_time", "id", "time"])
    weather_on_point["date"] = pd.to_datetime(weather_on_point["date"])
    df.day = pd.to_datetime(df.day)
    df = pd.merge(left=df, right=weather_on_point, left_on='day', right_on='date')
    fancy_string = str(hour) + "00"
    if len(fancy_string) == 3:
        fancy_string = "0" + fancy_string
    df = df.drop(columns=["date"]).rename(columns={
        "temperature": "temp-" + fancy_string,
        "condition": "cond-" + fancy_string
    })
    return df

def changed(from_hour, to_hour, df):
    w_from_hour = weather[weather["time"] == datetime.time(from_hour, 0)].drop(columns=["at_time", "id", "time"])
    w_from_hour["date"] = pd.to_datetime(w_from_hour["date"])
    w_to_hour = weather[weather["time"] == datetime.time(to_hour, 0)].drop(columns=["at_time", "id", "time"])
    w_to_hour["date"] = pd.to_datetime(w_to_hour["date"])
    col_string = "{}-{}".format(from_hour, to_hour)
    
    def do_something(row):
        first = w_from_hour[w_from_hour.date == row["day"]].iloc[0].condition
        second = w_to_hour[w_to_hour.date == row["day"]].iloc[0].condition
        return first == second
    
    df[colstring] = df.apply(do_something)
    return df

days = days.drop(columns=["delay", "vehicle"])
for i in range(0, 9):
    days = on_point(i, days)


In [75]:
# delays
mean_last_seen = pd.to_datetime(days["last_seen"]).mean()
mean_first_seen = pd.to_datetime(days["first_seen"]).mean()
print("mean 1st_seen:\t {}".format(mean_first_seen.time()))
print("mean last_seen:\t {}".format(mean_last_seen.time()))


days["first_delay"] = days.apply(lambda row: pd.to_datetime(row["first_seen"]) - mean_first_seen, axis=1)
print("first_delay type: {}".format(days["first_delay"].dtype))
# pd.to_datetime(days.last_seen).asType.plot.kde()

mean 1st_seen:	 08:11:59.999999
mean last_seen:	 08:28:22.857142
first_delay type: timedelta64[ns]


In [71]:
def columns_starting_with(string):
    return days[["day", "first_seen", "last_seen"] + [col for col in list(days.columns) if col.startswith(string)]]

days_conditions = columns_starting_with("cond")
days_temperaturs = columns_starting_with("temp")

In [73]:
days.columns

Index(['day', 'first_seen', 'last_seen', 'temp-0000', 'cond-0000', 'temp-0100',
       'cond-0100', 'temp-0200', 'cond-0200', 'temp-0300', 'cond-0300',
       'temp-0400', 'cond-0400', 'temp-0500', 'cond-0500', 'temp-0600',
       'cond-0600', 'temp-0700', 'cond-0700', 'temp-0800', 'cond-0800',
       'first_delay'],
      dtype='object')

In [51]:
days_conditions

,day,first_seen,last_seen,cond-0000,cond-0100,cond-0200,cond-0300,cond-0400,cond-0500,cond-0600,cond-0700,cond-0800
0,2019-09-09,08:06:00,08:06:00,cloudy,rainy,rainy,rainy,rainy,rainy,rainy,rainy,rainy
1,2019-09-10,08:12:00,08:30:00,rainy,rainy,rainy,cloudy,cloudy,cloudy,rainy,cloudy,clear
2,2019-09-11,08:06:00,08:18:00,clear,clear,clear,clear,clear,clear,clear,clear,clear
3,2019-09-12,08:12:00,08:30:00,rainy,rainy,rainy,rainy,rainy,rainy,cloudy,cloudy,cloudy
4,2019-09-13,08:06:00,08:18:00,clear,clear,clear,cloudy,cloudy,cloudy,cloudy,cloudy,cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...
58,2019-11-07,08:12:00,08:30:00,cloudy,cloudy,rainy,rainy,cloudy,cloudy,cloudy,cloudy,cloudy
59,2019-11-08,08:12:00,08:30:00,cloudy,cloudy,cloudy,clear,cloudy,clear,cloudy,cloudy,cloudy
60,2019-11-09,08:12:00,08:30:00,rainy,rainy,cloudy,rainy,rainy,rainy,rainy,rainy,rainy
61,2019-11-10,08:24:00,08:42:00,cloudy,cloudy,cloudy,cloudy,clear,clear,clear,clear,clear


In [52]:
days_temperaturs

,day,first_seen,last_seen,temp-0000,temp-0100,temp-0200,temp-0300,temp-0400,temp-0500,temp-0600,temp-0700,temp-0800
0,2019-09-09,08:06:00,08:06:00,14.6,14.4,14.1,13.2,13.1,13.3,13.4,13.6,13.6
1,2019-09-10,08:12:00,08:30:00,12.0,12.4,12.6,12.8,12.9,12.8,12.8,13.4,14.6
2,2019-09-11,08:06:00,08:18:00,8.7,7.9,7.3,6.7,6.4,6.4,8.0,11.1,14.5
3,2019-09-12,08:12:00,08:30:00,16.2,14.7,14.3,14.4,14.4,14.4,14.5,15.2,15.7
4,2019-09-13,08:06:00,08:18:00,13.9,13.6,13.7,14.0,14.7,15.6,16.3,17.5,18.8
...,...,...,...,...,...,...,...,...,...,...,...,...
58,2019-11-07,08:12:00,08:30:00,5.5,5.9,6.0,5.9,6.0,6.1,6.5,6.8,7.1
59,2019-11-08,08:12:00,08:30:00,7.1,6.2,5.4,4.4,3.7,3.6,3.2,3.2,4.9
60,2019-11-09,08:12:00,08:30:00,7.5,7.5,7.5,7.5,7.3,7.3,7.3,7.3,7.4
61,2019-11-10,08:24:00,08:42:00,4.5,4.7,4.7,4.0,2.6,2.0,2.2,2.4,2.5


In [ ]:
days.to_csv("days.csv", sep=";", decimal=",")